0. Scan glass slides at 20x
1. Create Excel sheet with patient information
2. Screen H&E images to select potentially healthy sections
3. Annotate the Region of Interest (ROI) (Optional)
4. Convert ROI XML to Image (Code: roixml2png.py)
5. Convert H&E images to 1um resolution tif to apply DeepLab (Code: rescale_wsi.py) (Matlab Code: infer_12class)
6. Annotate 12 tissue compartments, apply DeepLab, and evaluate performance metrics
7. Apply HoVerNet
8. Rotate and crop tissue pieces, DLmask, HoVerNet segmented nuclei mask (Code: align_he_dl_cnt.py)
9. Calculate tissue composition (Code: DLcomposition.py)
10. Calculate epidermis thickness, waviness, roughness (Matlab Code)
12. Calculate dermis thickness, waviness (Matlab Code)
13. Calculate ECM anisotropy,Interstitial orientation, Thick gap ratio, Thick gap average (Matlab Code)
14. Calculate tissue morphology (Matlab Code)
15. ECM cell count and morphology + std using Manual Threshold method
16. Distance from ECM cell to tissue
17. All celltype morphology, orientation, and distance to tissue using HoVerNet (Code: hovernet_json2df + summarize_nuclei_shape_df ; these codes are unrotated; need update if we want to add orientation  or postprocess DLmask)
18. Further add features such as more detailed cell subtyping (TBD)

In [1]:
import os
import pandas as pd
from natsort import natsorted
from PIL import Image
from matplotlib import pyplot as plt

#utilities
from roixml2png import roixml2png
from rescale_wsi import rescale_wsi
from align_he_dl_cnt import align_he_dl_cnt
from DLcomposition import DLcomposition

In [21]:
wsisrc = r'\\fatherserverdw\kyuex\clue images'
oneum = r'\\fatherserverdw\kyuex\clue images\1um'
aligned_images_dst = r'\\fatherserverdw\kyuex\datadst\20220929'
imcropsrc = os.path.join(aligned_images_dst,'imcrop')
imcroproi = os.path.join(aligned_images_dst,'imcrop_roi')
dlsrc = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dlcropsrc = os.path.join(aligned_images_dst,'dlcrop')
dlcroproi = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop_roi'
roixml = r'\\fatherserverdw\kyuex\clue images\annotations\roi'
roisrc = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
tissue_area_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\TA_20rsf'
twelve_annot = r'\\fatherserverdw\kyuex\clue images\annotations\12class'
LUT = r"\\fatherserverdw\kyuex\imlist_all.xlsx"
cntsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
analysisdst= r'\\fatherserverdw\kyuex\clue images\analysis output'

In [3]:
dlcropsrc

'\\\\fatherserverdw\\kyuex\\datadst\\20220929\\dlcrop'

In [4]:
run_roixml2png = False
run_rescale_wsi = False
run_align_he_dl_cnt = False
run_DLcomposition = True

In [5]:
#Load healthy tissue look up table (LUT)
LUT = pd.read_excel(LUT)
# xmlist = LUT['filename'][(LUT['student score']>1) & (LUT['Block or Slide?']=="Both")]
LUThealthy = LUT[(LUT['student score']>1)]
LUThealthyback = LUThealthy[LUThealthy['body part 1'].str.lower()=="back"]

In [6]:
#Step 4: ROI XML 2 Image
#Saves Image to where XML is in subfolders
if run_roixml2png:
    xmlist = [os.path.join(roixml,xmlpth.replace('ndpi','xml')) for xmlpth in LUThealthyback['filename']]
    [roixml2png(xmlpth,wsisrc) for xmlpth in xmlist]
    # pd.DataFrame(np.array(roiinfo),columns=['fn','ROIA','TA','ratio']).to_csv(r"\\fatherserverdw\kyuex\ROITA_ratio.csv")

In [7]:
#Step 5: rescale wsi to 1um for applying deeplab
#Saves Image to where wsi is in subfolder
if run_rescale_wsi:
    wsis = [os.path.join(wsisrc,_) for _ in LUThealthy.filename]
    wsis = natsorted(wsis)
    for idx,wsi in enumerate(wsis):
        rescale_wsi(wsi,1)
        print(idx,'/',len(wsis))

In [8]:
# Narrow down LUT to which that has json, dlmask, roi
def hasjson(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(cntsrc,fn+'.json'))
def hasdl(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(dlcropsrc,fn+'.tif'))
def hasroi(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(roisrc,fn+'.png'))
LUThealthyback['hasjson']=LUThealthyback['filename'].apply(lambda row: hasjson(row))
LUThealthyback['hasdl']=LUThealthyback['filename'].apply(lambda row: hasdl(row))
LUThealthyback['hasroi']=LUThealthyback['filename'].apply(lambda row: hasroi(row))
LUThealthyback2 = LUThealthyback[(LUThealthyback['hasjson']==True) & (LUThealthyback['hasdl']==True) & (LUThealthyback['hasroi']==True)]
LUThealthyback2 = LUThealthyback2.reset_index()
len(LUThealthyback2),len(LUThealthyback[(LUThealthyback['hasjson']==True)]),len(LUThealthyback[(LUThealthyback['hasdl']==True)]),len(LUThealthyback)

C:\Users\kyuha\AppData\Local\Temp\ipykernel_17032\1105240623.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LUThealthyback['hasjson']=LUThealthyback['filename'].apply(lambda row: hasjson(row))
C:\Users\kyuha\AppData\Local\Temp\ipykernel_17032\1105240623.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LUThealthyback['hasdl']=LUThealthyback['filename'].apply(lambda row: hasdl(row))
C:\Users\kyuha\AppData\Local\Temp\ipykernel_17032\1105240623.py:16: SettingWithCopyWarning: 
A value is trying to be s

(53, 53, 66, 66)

In [9]:
# Step 8: Rotate and crop tissue pieces, DLmask, HoVerNet segmented nuclei mask (Code: align_he_dl_cnt.py)
if run_align_he_dl_cnt:
    for idx,row in LUThealthyback2.iterrows():
        print(idx,'/',len(LUThealthyback2))
        fn,ext = os.path.splitext(row['filename'])
        try:
            rotationdf = align_he_dl_cnt(aligned_images_dst,fn,wsisrc,dlsrc,cntsrc,roisrc)
        except:
            print(fn)

In [10]:
# Narrow down LUT to which that has rotated image crop and dl crop
def hasimcrop(row):
    fn,ext = os.path.splitext(row)
    fn = fn+'sec1'
    return os.path.exists(os.path.join(imcropsrc,fn+'.png'))
LUThealthyback2['hasimcrop']=LUThealthyback2['filename'].apply(lambda row: hasimcrop(row))
LUThealthyback3=LUThealthyback2[LUThealthyback2['hasimcrop']]
LUThealthyback3 = LUThealthyback3.reset_index(drop=True)

In [31]:
def hasdlcrop(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(dlcropsrc,fn+'sec1.png'))
LUThealthy['hasdlcrop']=LUThealthy['filename'].apply(lambda row: hasdlcrop(row))
LUThealthy2 = LUThealthy[(LUThealthy['hasdlcrop']==True)].reset_index(drop=True)
if run_DLcomposition:
    fns = []
    comps = []
    secs = []
    for idx,row in LUThealthy2.iterrows():
        print(idx+1,'/',len(LUThealthy2))
        fn,ext = os.path.splitext(row['filename'])
        sections = [_ for _ in os.listdir(dlcropsrc) if fn in _]
        for section in sections:
            comp = DLcomposition(os.path.join(dlcropsrc,section))
            comps.append(comp)
            fn = row['filename']
            fns.append(fn)
            sec = section[-5]
            secs.append(sec)

1 / 63
2 / 63
3 / 63
4 / 63
5 / 63
6 / 63
7 / 63
8 / 63
9 / 63
10 / 63
11 / 63
12 / 63
13 / 63
14 / 63
15 / 63
16 / 63
17 / 63
18 / 63
19 / 63
20 / 63
21 / 63
22 / 63
23 / 63
24 / 63
25 / 63
26 / 63
27 / 63
28 / 63
29 / 63
30 / 63
31 / 63
32 / 63
33 / 63
34 / 63
35 / 63
36 / 63
37 / 63
38 / 63
39 / 63
40 / 63
41 / 63
42 / 63
43 / 63
44 / 63
45 / 63
46 / 63
47 / 63
48 / 63
49 / 63
50 / 63
51 / 63
52 / 63
53 / 63
54 / 63
55 / 63
56 / 63
57 / 63
58 / 63
59 / 63
60 / 63
61 / 63
62 / 63
63 / 63


In [30]:
pd.DataFrame([fns,secs]).T.to_csv(os.path.join(analysisdst,'composition.csv'))

In [32]:
pd.concat([pd.DataFrame([fns,secs]).T,pd.DataFrame(comps)],axis=1).to_csv(os.path.join(analysisdst,'composition.csv'))

In [66]:
a=pd.read_csv(r"\\fatherserverdw\kyuex\clue images\analysis output\avgdf.csv")
b=pd.read_csv(r"\\fatherserverdw\kyuex\clue images\analysis output\complete_datasheet\epidermis_profile.csv",header=3)
def isin2(row):
    return b['filename'].str.contains(row+'.ndpi').any()
a['isin']=a['imID'].apply(lambda row:isin2(row))
a.to_csv(r"\\fatherserverdw\kyuex\clue images\analysis output\avgdf.csv")

In [67]:
a=pd.read_csv(r"\\fatherserverdw\kyuex\clue images\analysis output\cvdf.csv")
b=pd.read_csv(r"\\fatherserverdw\kyuex\clue images\analysis output\complete_datasheet\epidermis_profile.csv",header=3)
def isin2(row):
    return b['filename'].str.contains(row+'.ndpi').any()
a['isin']=a['imID'].apply(lambda row:isin2(row))
a.to_csv(r"\\fatherserverdw\kyuex\clue images\analysis output\cvdf.csv")

In [62]:
b=pd.read_csv(r"\\fatherserverdw\kyuex\clue images\analysis output\complete_datasheet\epidermis_profile.csv",header=3)
b

,index,gender,age,pid,sectionID,filename,year,sampleId,labelName,subLabel,...,stromal_nucleus2follicle_CV,stromal_nucleus2muscle_CV,stromal_nucleus2oil_CV,stromal_nucleus2sweat_CV,stromal_nucleus2nerve_CV,stromal_nucleus2blood_CV,stromal_nucleus2ecm_CV,stromal_nucleus2fat_CV,stromal_Ncell_CV,stromal_Ncell_CV.1
0,1143,F,85.604454,471,1,2022-06-07 14.34.59.ndpi,10,3702,10_3702,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1144,F,85.604454,471,1,2022-06-07 14.37.19.ndpi,10,3702,10_3702,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1173,M,92.435616,484,1,2022-06-07 17.24.15.ndpi,10,6634,10_6634,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,162,M,74.745205,71,1,2022-06-07 17.43.53.ndpi,3,18469,3_18469,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,394,M,87.602740,178,1,2022-06-08 17.04.20.ndpi,5,9867,5_9867,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,293,F,72.603825,133,1,2022-07-07 19.19.08.ndpi,4,15023,4_15023,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,402,M,87.602740,178,1,2022-07-07 20.21.33.ndpi,5,9867,5_9867,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,329,M,55.600000,151,1,2022-07-07 21.21.09.ndpi,5,3647,5_3647,B4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,328,M,55.600000,151,1,2022-07-07 21.27.08.ndpi,5,3647,5_3647,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
